# Tensorflow Object Detection API - Tutorial
This tutorial serves as an introduction to the basic workflows surrounding the use of the most popular research model in Tensorflow, the Object Detection API. Here we go through all the steps required to setup a development environment for assembling a dataset, preparing the input files, training detection models and running data through them. We demonstrate all the above by using the Oxford-IIIT Pet Dataset.

## Environment setup

Check the GPU type assigned to your instance.

In [ ]:
!nvidia-smi

Browse information about the instance's CPU.

In [ ]:
!lscpu

## Installation - Dependencies

In [ ]:
use_my_drive = False

if use_my_drive:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=False)

    from os import chdir
    chdir("/content/drive/My Drive/")

In [ ]:
!pwd

In [ ]:
!git clone https://github.com/tensorflow/models

In [ ]:
%%bash
cd models
git reset --hard 126ce65
rm -rf .git

In [ ]:
!pip install -U --pre tensorflow=="2.3.0"
!pip install tf_slim

In [ ]:
!apt-get install git protobuf-compiler python3-pil python3-lxml python3-tk
!pip install --user Cython
!pip install --user contextlib2
!pip install --user jupyter
!pip install --user matplotlib

In [ ]:
!pip install --user pycocotools

In [ ]:
%cd models/research/

In [ ]:
!protoc object_detection/protos/*.proto --python_out=.

In [ ]:
!pip install .

In [ ]:
%env PYTHONPATH=/env/python:/content/models:/content/models/research/slim

In [ ]:
import sys
sys.path.append('/content/models')

In [ ]:
!python object_detection/builders/model_builder_tf2_test.py

## Data preparation

In [ ]:
!mkdir data data/tfrecords

In [ ]:
!wget http://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz
!wget http://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz
!tar -xvf images.tar.gz
!tar -xvf annotations.tar.gz
!rm -rf images.tar.gz
!rm -rf annotations.tar.gz
!mv images data/
!mv annotations data/

In [ ]:
!git clone https://github.com/johntikas/pet-detection

In [ ]:
!cp -avr pet-detection/images .

In [ ]:
!cp -avr pet-detection/xmls annotations/

In [ ]:
!python object_detection/dataset_tools/create_pet_tf_record.py \
 --label_map_path=object_detection/data/pet_label_map.pbtxt \
 --faces_only=False \
 --data_dir=data \
 --output_dir=data/tfrecords

In [ ]:
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8.tar.gz
!tar -xvf mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8.tar.gz
!rm -rf mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8.tar.gz

## Model - Training

In [ ]:
!python object_detection/model_main_tf2.py \
--logtostderr \
--pipeline_config_path=pet-detection/configs/mask_rcnn_inception_resnet_v2_pets.config \
--model_dir=model

## Model - Inference

In [ ]:
!python object_detection/exporter_main_v2.py \
--input_type=image_tensor \
--pipeline_config_path=pet-detection/configs/mask_rcnn_inception_resnet_v2_pets.config \
--trained_checkpoint_dir=model \
--output_directory=model/export

In [ ]:
import tensorflow as tf

import matplotlib
import matplotlib.pyplot as plt

import io
import os
import pathlib
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

%matplotlib inline

In [ ]:
def load_image_into_numpy_array(path):
    img_data = tf.io.gfile.GFile(path, 'rb').read()
    image = Image.open(BytesIO(img_data))
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

def get_keypoint_tuples(eval_config):
    tuple_list = []
    kp_list = eval_config.keypoint_edge
    for edge in kp_list:
      tuple_list.append((edge.start, edge.end))
    return tuple_list

In [ ]:
current_dir = os.getcwd()
config_dir = '/pet-detection/configs/mask_rcnn_inception_resnet_v2_pets.config'
pipeline_config = os.path.join(current_dir + config_dir)
model_dir = current_dir + 'model/'

# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(pipeline_config)
model_config = configs['model']
detection_model = model_builder.build(
      model_config=model_config, is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
# ckpt.restore(os.path.join(model_dir, 'ckpt-0')).expect_partial()

def get_model_detection_function(model):
  @tf.function
  def detect_fn(image):
    image, shapes = model.preprocess(image)
    prediction_dict = model.predict(image, shapes)
    detections = model.postprocess(prediction_dict, shapes)
    return detections, prediction_dict, tf.reshape(shapes, [-1])
  return detect_fn

detect_fn = get_model_detection_function(detection_model)

In [ ]:
label_map_path = configs['eval_input_config'].label_map_path
label_map = label_map_util.load_labelmap(label_map_path)
categories = label_map_util.convert_label_map_to_categories(
    label_map,
    max_num_classes=label_map_util.get_max_label_map_index(label_map),
    use_display_name=True)
category_index = label_map_util.create_category_index(categories)
label_map_dict = label_map_util.get_label_map_dict(label_map, use_display_name=False)

In [ ]:
image_dir = '/content/models/research/data/images/'
image_path = os.path.join(image_dir, 'Abyssinian_100.jpg')
image_np = load_image_into_numpy_array(image_path)
input_tensor = tf.convert_to_tensor(
    np.expand_dims(image_np, 0), dtype=tf.float32)
detections, predictions_dict, shapes = detect_fn(input_tensor)

label_id_offset = 1
image_np_with_detections = image_np.copy()

# Use keypoints if available in detections
keypoints, keypoint_scores = None, None
if 'detection_keypoints' in detections:
  keypoints = detections['detection_keypoints'][0].numpy()
  keypoint_scores = detections['detection_keypoint_scores'][0].numpy()

viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np_with_detections,
      detections['detection_boxes'][0].numpy(),
      (detections['detection_classes'][0].numpy() + label_id_offset).astype(int),
      detections['detection_scores'][0].numpy(),
      category_index,
      use_normalized_coordinates=True,
      max_boxes_to_draw=200,
      min_score_thresh=.30,
      agnostic_mode=False,
      keypoints=keypoints,
      keypoint_scores=keypoint_scores,
      keypoint_edges=get_keypoint_tuples(configs['eval_config']))

plt.figure(figsize=(12,16))
plt.imshow(image_np_with_detections)
plt.show()